In [165]:
import hashlib
import time
import random
import string

### Defines a wallet with a public and private key and balance

In [225]:
class Wallet:
    def __init__(self, name):
        # Generate a random private and public key pair for the wallet
        self.name = name
        self.private_key = ''.join(random.choices(string.ascii_letters + string.digits, k=32))
        self.public_key = hashlib.sha256(self.private_key.encode()).hexdigest()
        self.balance = 0

    def add_balance(self, amount):
        self.balance += amount
        return self
    
    def display_all_wallet(self):
        print(f"Wallet {self.name}:")
        print(f"  Public Key: {self.public_key}")
        print(f"  Private Key: {self.private_key}")
        print(f"  Balance: {self.balance} units")

    def display_public_key(self):
        print(f"Public Key: {self.public_key}")

    def display_private_key(self):
        print(f"Private Key: {self.private_key}")

### Defines the transaction sender, recipient, amount, time, id

In [226]:
class Transaction:
    def __init__(self, sender, recipient, amount):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.timestamp = time.time()
        self.transaction_id = self.calculate_transaction_id()

    def calculate_transaction_id(self):
        # Hash the transaction details to create a unique identifier
        data = f"{self.sender.public_key if self.sender else 'None'}{self.recipient.public_key}{self.amount}{self.timestamp}"
        return hashlib.sha256(data.encode()).hexdigest()

    def execute_transaction(self):
        if self.sender and self.sender.balance < self.amount:
            raise ValueError("Insufficient funds for the transaction.")

        # Deduct the amount from the sender's balance (if sender is not None)
        if self.sender:
            self.sender.balance -= self.amount

        # Add the amount to the recipient's balance
        self.recipient.balance += self.amount


    def display_transaction(self):
        formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(self.timestamp))
        print("Transaction:")
        print(f"  Sender Public Key: {self.sender.public_key if self.sender else 'None'}")
        print(f"  Recipient Public Key: {self.recipient.public_key}")
        print(f"  Amount: {self.amount}")
        print(f"  Timestamp: {formatted_timestamp}")
        print(f"  Transaction ID: {self.transaction_id}")

### Defines the Block which has a number, timestamp, transaction data, previous hash and its hash

In [227]:
class Block:
    def __init__(self, block_number, previous_hash, transactions):
        self.block_number = block_number
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        data = (
            str(self.block_number)
            + str(self.timestamp)
            + str([transaction.transaction_id for transaction in self.transactions])
            + str(self.previous_hash)
        )
        return hashlib.sha256(data.encode()).hexdigest()

### Defines the blockchain Creates the first block when called and then has a function to add_blocks (in a real blockchain the Blockchain.chain would be immutable but in this case it isn't)

In [230]:
class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.current_transactions = []

    def create_genesis_block(self):
        # Create the first block in the chain (Genesis Block)
        return Block(0, "0", [])

    def mine_block(self, miner_wallet, miner_reward):
        # Create a new block with the pending transactions
        new_block = Block(len(self.chain), self.chain[-1].hash, self.current_transactions)
        # Reset the list of pending transactions
        self.current_transactions = []

        # Reward the miner for mining a new block
        miner_transaction = Transaction(None, miner_wallet, miner_reward)
        miner_transaction.execute_transaction()

        # Execute all transactions in the mined block
        for transaction in new_block.transactions:
            transaction.execute_transaction()

        # Add the new block to the blockchain
        self.chain.append(new_block)

    def add_transaction(self, transaction):
        # Ensure that the transaction is valid before adding it to the blockchain
        if not isinstance(transaction, Transaction):
            raise ValueError("Invalid transaction. Must be an instance of the Transaction class.")

        # Execute the transaction and add it to the pending transactions
        transaction.execute_transaction()
        self.current_transactions.append(transaction)

    def display_chain(self):
        for block in self.chain:
            formatted_timestamp = time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(block.timestamp))
            transaction_ids = [transaction.transaction_id for transaction in block.transactions]
            print(f"Block #{block.block_number} | Hash: {block.hash}")
            print(f"Timestamp: {formatted_timestamp}")
            print(f"Transaction IDs: {transaction_ids}")
            print("--------------")

### Mining

--------------
1. Transaction: Transfer 10 BTC from Alice to Bob
2. Transaction: Transfer 5 BTC from Bob to Charlie
3. Transaction: Transfer 2 BTC from Alice to Eve
--------------

--------------
1. Alice(20) -10-> bob, Alice -2-> Eve
2. Bob(20) -5-> Charlie
3. Charlie(20) 
4. Eve(20)
--------------
Result:
Alice(8)
Bob(25)
Charlie(25)
Eve(22)

In [232]:
if __name__ == "__main__":
    # Create wallets
    alice_wallet = Wallet("Alice")
    bob_wallet = Wallet("Bob")
    charlie_wallet = Wallet("Charlie")
    miner_wallet = Wallet("Miner")

    # Initial balances
    alice_wallet.add_balance(100)
    bob_wallet.add_balance(50)
    charlie_wallet.add_balance(30)

# Display initial balances
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: d76725354c1287538c95dc0898051f22d8a3544a8f349733f01f314bd4320eea
  Private Key: wUvfo1PjsLoASU7Z3vBx8RYCTduVRxQT
  Balance: 100 units
Wallet Bob:
  Public Key: 43513515edbd55a5854727fc1983b45f56cea86592b5c3e257605e618d9f83a5
  Private Key: xAIS5VhcxSeP4WZaNAdG4iXIsuRbf0eC
  Balance: 50 units
Wallet Charlie:
  Public Key: ac3575814ec0152d2907fe9dc2d5eef6d919aee5caeed2ee65fa6d855b69ff0f
  Private Key: BwlyhlbZiiWUEA96zb8okfDjEjJQV7jx
  Balance: 30 units


In [233]:
# Create a blockchain
my_blockchain = Blockchain()

In [234]:
my_blockchain.display_chain()

Block #0 | Hash: c306b870dad883528ddb60c395f16bffe0ea46dd9c30a3e9ac8fb29e82552517
Timestamp: 12/05/2023 02:04:44
Transaction IDs: []
--------------


In [235]:
# Create transactions
transaction1 = Transaction(alice_wallet, bob_wallet, 10)
transaction2 = Transaction(bob_wallet, charlie_wallet, 5)
transaction3 = Transaction(charlie_wallet, alice_wallet, 8)

# Add transactions to the blockchain
my_blockchain.add_transaction(transaction1)
my_blockchain.add_transaction(transaction2)
my_blockchain.add_transaction(transaction3)

Block #0 | Hash: c306b870dad883528ddb60c395f16bffe0ea46dd9c30a3e9ac8fb29e82552517
Timestamp: 12/05/2023 02:04:44
Transaction IDs: []
--------------


In [237]:
transaction1.display_transaction()
transaction2.display_transaction()
transaction3.display_transaction()

Transaction:
  Sender Public Key: d76725354c1287538c95dc0898051f22d8a3544a8f349733f01f314bd4320eea
  Recipient Public Key: 43513515edbd55a5854727fc1983b45f56cea86592b5c3e257605e618d9f83a5
  Amount: 10
  Timestamp: 12/05/2023 02:05:18
  Transaction ID: 35fdd92f7d958431e98b90f3d2879109ab7c3c42401c4168037a5db435b6b172
Transaction:
  Sender Public Key: 43513515edbd55a5854727fc1983b45f56cea86592b5c3e257605e618d9f83a5
  Recipient Public Key: ac3575814ec0152d2907fe9dc2d5eef6d919aee5caeed2ee65fa6d855b69ff0f
  Amount: 5
  Timestamp: 12/05/2023 02:05:18
  Transaction ID: 03a5cb394f8afc7c4ae417f2a7f54f3657b623bdb9e166e33218df45e196184d
Transaction:
  Sender Public Key: ac3575814ec0152d2907fe9dc2d5eef6d919aee5caeed2ee65fa6d855b69ff0f
  Recipient Public Key: d76725354c1287538c95dc0898051f22d8a3544a8f349733f01f314bd4320eea
  Amount: 8
  Timestamp: 12/05/2023 02:05:18
  Transaction ID: 4514d1db2a1f602e7ce3c577084b184cfd5fb17b48aa38044055b87573f1eb68


In [238]:
alice_wallet.display_all_wallet()
bob_wallet.display_all_wallet()
charlie_wallet.display_all_wallet()

Wallet Alice:
  Public Key: d76725354c1287538c95dc0898051f22d8a3544a8f349733f01f314bd4320eea
  Private Key: wUvfo1PjsLoASU7Z3vBx8RYCTduVRxQT
  Balance: 98 units
Wallet Bob:
  Public Key: 43513515edbd55a5854727fc1983b45f56cea86592b5c3e257605e618d9f83a5
  Private Key: xAIS5VhcxSeP4WZaNAdG4iXIsuRbf0eC
  Balance: 55 units
Wallet Charlie:
  Public Key: ac3575814ec0152d2907fe9dc2d5eef6d919aee5caeed2ee65fa6d855b69ff0f
  Private Key: BwlyhlbZiiWUEA96zb8okfDjEjJQV7jx
  Balance: 27 units


In [239]:
# Mine a block with a reward for the miner
miner_reward = 5
my_blockchain.mine_block(miner_wallet, miner_reward)

# Display the blockchain
my_blockchain.display_chain()

Block #0 | Hash: c306b870dad883528ddb60c395f16bffe0ea46dd9c30a3e9ac8fb29e82552517
Timestamp: 12/05/2023 02:04:44
Transaction IDs: []
--------------
Block #1 | Hash: 8d20fede74633057e0db858c7530e967bd2e6593c75442ac5a96c4a632180ed0
Timestamp: 12/05/2023 02:07:42
Transaction IDs: ['35fdd92f7d958431e98b90f3d2879109ab7c3c42401c4168037a5db435b6b172', '03a5cb394f8afc7c4ae417f2a7f54f3657b623bdb9e166e33218df45e196184d', '4514d1db2a1f602e7ce3c577084b184cfd5fb17b48aa38044055b87573f1eb68']
--------------
